# Clustering Crypto

In [59]:
# Initial imports
import pandas as pd
import hvplot.pandas
from pathlib import Path
import plotly.express as px
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans


### Deliverable 1: Preprocessing the Data for PCA

In [60]:
# Load the crypto_data.csv dataset.
path = Path("../Challenge_18/crypto_data.csv")
crypto_df  =pd.read_csv(path)
crypto_df

,Unnamed: 0,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,True,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,True,PoW,NaN,611000
4,808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...,...
1247,XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
1248,DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
1249,GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
1250,OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [61]:
# Keep all the cryptocurrencies that are being traded.
a = crypto_df[crypto_df["IsTrading"] == False].index
crypto_df.drop(a, inplace=True)

crypto_df.count()

Unnamed: 0         1144
CoinName           1144
Algorithm          1144
IsTrading          1144
ProofType          1144
TotalCoinsMined     685
TotalCoinSupply    1144
dtype: int64

In [62]:
#Drop the IsTrading column
crypto_df.drop(columns=["IsTrading"], inplace=True)
crypto_df

,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
1,365,365Coin,X11,PoW/PoS,NaN,2300000000
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
3,611,SixEleven,SHA-256,PoW,NaN,611000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
1243,SERO,Super Zero,Ethash,PoW,NaN,1000000000
1244,UOS,UOS,SHA-256,DPoI,NaN,1000000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [63]:
#Remove rows that have at least one null value.

crypto_df = crypto_df.dropna()
crypto_df


,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
4,808,808,SHA-256,PoW/PoS,0.000000e+00,0
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [73]:
# Keep the rows where coins are mined.
crypto_df = crypto_df[crypto_df["TotalCoinsMined"] > 0]
crypto_df


,Unnamed: 0,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
0,42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
2,404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
5,1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
7,BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
8,ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...,...
1238,ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
1242,GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
1245,BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
1246,ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [87]:
crypto_df= crypto_df.rename(columns={"Unnamed: 0":"Index"})
crypto_df


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610


In [95]:
crypto_df.index


Index(['42', '404', '1337', 'BTC', 'ETH', 'LTC', 'DASH', 'XMR', 'ETC', 'ZEC',
       ...
       'TCH', 'WAVES', 'PART', 'BTT', 'NXT', 'ZEPH', 'GAP', 'BDX', 'ZEN',
       'XBC'],
      dtype='object', name='Unnamed: 0', length=532)

In [ ]:
# Create a new DataFrame that holds only the cryptocurrencies names.
# YOUR CODE HERE

In [103]:
crypto_df.head()


,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,,
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0


In [104]:
crypto_df_names = crypto_df[["CoinName"]]

In [106]:
crypto_df_names.head()

,CoinName
Unnamed: 0,
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum


In [111]:
# Drop the 'CoinName' column since it's not going to be used on the clustering algorithm.

crypto_df = crypto_df.drop(["CoinName"],axis=1)

In [112]:
crypto_df.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
Unnamed: 0,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0


In [ ]:
# Use get_dummies() to create variables for text features.

In [227]:
X = pd.get_dummies(crypto_df, columns=["Algorithm","ProofType"])
X.head(10)

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
LTC,6.303924e+07,84000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
DASH,9.031294e+06,22000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
XMR,1.720114e+07,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETC,1.133597e+08,210000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [228]:

X_scaled = StandardScaler().fit_transform(X)
print(X_scaled[0:5])


[[-0.11710817 -0.1528703  -0.0433963  -0.0433963  -0.0433963  -0.06142951
  -0.07530656 -0.0433963  -0.06142951 -0.06142951 -0.0433963  -0.0433963
  -0.19245009 -0.06142951 -0.09740465 -0.0433963  -0.11547005 -0.07530656
  -0.0433963  -0.0433963  -0.15191091 -0.0433963  -0.13118084 -0.0433963
  -0.0433963  -0.08703883 -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.06142951 -0.0433963  -0.08703883 -0.08703883 -0.08703883 -0.0433963
  -0.13118084 -0.13840913 -0.13840913 -0.0433963  -0.06142951 -0.0433963
  -0.07530656 -0.18168574 -0.0433963  -0.0433963  -0.0433963  -0.07530656
  -0.15826614 -0.31491833 -0.0433963  -0.08703883 -0.07530656 -0.06142951
   1.38675049 -0.0433963  -0.0433963  -0.06142951 -0.0433963  -0.0433963
  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963  -0.0433963
  -0.39879994 -0.0433963  -0.18168574 -0.0433963  -0.08703883 -0.08703883
  -0.10680283 -0.0433963  -0.13118084 -0.0433963  -0.0433963  -0.0433963
  -0.0433963  -0.07530656 -0.43911856 -0.04339

## Deliverable 2: Reducing Data Dimensions Using PCA

In [229]:
#initialize PCA model
pca = PCA(n_components=3)

In [230]:
#PCA Training
X_pca = pca.fit_transform(X_scaled)

In [231]:
pcs_df = pd.DataFrame(
    data=X_pca, columns=["PC1", "PC2", "PC3"], index =crypto_df.index)
pcs_df.head()

,PC1,PC2,PC3
Unnamed: 0,,,
42,-0.326982,1.038602,-0.569113
404,-0.310282,1.038649,-0.569433
1337,2.329207,1.650347,-0.631610
BTC,-0.145800,-1.315161,0.203686
ETH,-0.153792,-2.077139,0.323117


### Deliverable 3: Clustering Crytocurrencies Using K-Means

#### Finding the Best Value for `k` Using the Elbow Curve

In [232]:
# Create an elbow curve to find the best value for K.
# YOUR CODE HERE


#find the best value for K 
inertia = []
k = list(range(1,11))
for i in k:
    km =KMeans(n_clusters=i, random_state=0)
    km.fit(pcs_df)
    inertia.append(km.inertia_)
    

#create the elbow curve
elbow_data = {"k":k,"inertia":inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title = "ELBOW CURVE")


C:\ProgramData\Anaconda3\envs\mlenv\lib\site-packages\sklearn\cluster\_kmeans.py:1037: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=3.
  "KMeans is known to have a memory leak on Windows "


:Curve   [k]   (inertia)

In [244]:
# Initialize the K-Means model.

model = KMeans(n_clusters=4,random_state=2)

# Fit the model
model.fit(pcs_df)

# Predict clusters
prediction = model.predict(pcs_df)
prediction

array([0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1,
       0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0,

In [262]:
# Create a new DataFrame including predicted clusters and cryptocurrencies features.
# Concatentate the crypto_df and pcs_df DataFrames on the same columns.

data = [crypto_df, pcs_df]
clustered_df2 = pd.concat([crypto_df, pcs_df], join="inner", axis=1)

In [263]:
clustered_df2.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3
Unnamed: 0,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.326982,1.038602,-0.569113
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.310282,1.038649,-0.569433
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.329207,1.650347,-0.631610
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.145800,-1.315161,0.203686
ETH,Ethash,PoW,1.076842e+08,0,-0.153792,-2.077139,0.323117


In [264]:
#  Add a new column, "CoinName" to the clustered_df DataFrame that holds the names of the cryptocurrencies. 
clustered_df2 = clustered_df2.merge(crypto_df_names, left_index=True, right_index=True)

In [265]:
#  Add a new column, "Class" to the clustered_df DataFrame that holds the predictions
clustered_df2["class"] = model.labels_

In [266]:
clustered_df2.head()

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,class
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.326982,1.038602,-0.569113,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.310282,1.038649,-0.569433,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.329207,1.650347,-0.631610,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.145800,-1.315161,0.203686,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.153792,-2.077139,0.323117,Ethereum,1


In [267]:
# Print the shape of the clustered_df
print(clustered_df2.shape)
clustered_df2.head(10)

(532, 9)


,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,class
Unnamed: 0,,,,,,,,,
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.326982,1.038602,-0.569113,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.310282,1.038649,-0.569433,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.329207,1.650347,-0.631610,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.145800,-1.315161,0.203686,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.153792,-2.077139,0.323117,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.168840,-1.111427,-0.024461,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.380428,1.174957,-0.500135,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.156233,-2.206639,0.360001,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.152230,-2.077254,0.323103,Ethereum Classic,1


### Deliverable 4: Visualizing Cryptocurrencies Results

#### 3D-Scatter with Clusters

In [298]:
# Creating a 3D-Scatter with the PCA data and the clusters

fig = px.scatter_3d(clustered_df2, x="PC1", y="PC2", z="PC3", symbol="class", width=800, color="class",hover_name="CoinName")
fig.show()


In [300]:
# Create a table with tradable cryptocurrencies.
clustered_df2.hvplot.table()

:Table   [Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC1,PC2,PC3,CoinName,class]

In [307]:
# Print the total number of tradable cryptocurrencies.

print(f'There are total number of {clustered_df2["class"].count()} tradable Cryptocurrencies.')


There are total number of 532 tradable Cryptocurrencies.


In [311]:
# Scaling data to create the scatter plot with tradable cryptocurrencies.
X_supplied_mined = clustered_df2[["TotalCoinSupply","TotalCoinsMined"]]
X_supplied_mined

,TotalCoinSupply,TotalCoinsMined
Unnamed: 0,,
42,42,4.199995e+01
404,532000000,1.055185e+09
1337,314159265359,2.927942e+10
BTC,21000000,1.792718e+07
ETH,0,1.076842e+08
...,...,...
ZEPH,2000000000,2.000000e+09
GAP,250000000,1.493105e+07
BDX,1400222610,9.802226e+08


In [326]:
#scale dataset using MinMaxScaler()
X_supplied_mined_scaled = MinMaxScaler().fit_transform(X_supplied_mined)
X_supplied_mined_scaled

array([[4.20000000e-11, 0.00000000e+00],
       [5.32000000e-04, 1.06585544e-03],
       [3.14159265e-01, 2.95755135e-02],
       ...,
       [1.40022261e-03, 9.90135079e-04],
       [2.10000000e-05, 7.37028150e-06],
       [1.00000000e-06, 1.29582282e-07]])

In [318]:
clustered_df3 =[["X_supplied_mined_scaled", "clustered_df2"]]

In [356]:
X_supplied_mined_scaled_df = pd.DataFrame(X_supplied_mined_scaled, columns=["TotalCoinSupply","TotalCoinsMined"], index=clustered_df2.index)

In [357]:
X_supplied_mined_scaled_df

,TotalCoinSupply,TotalCoinsMined
Unnamed: 0,,
42,4.200000e-11,0.000000e+00
404,5.320000e-04,1.065855e-03
1337,3.141593e-01,2.957551e-02
BTC,2.100000e-05,1.810842e-05
ETH,0.000000e+00,1.087731e-04
...,...,...
ZEPH,2.000000e-03,2.020225e-03
GAP,2.500000e-04,1.508199e-05
BDX,1.400223e-03,9.901351e-04


In [358]:
X_supplied_mined_scaled_df = X_supplied_mined_scaled_df.merge(crypto_df_names, left_index=True, right_index=True)

In [361]:
X_supplied_mined_scaled_df["class"] = model.labels_ 
X_supplied_mined_scaled_df

,TotalCoinSupply,TotalCoinsMined,CoinName,class
Unnamed: 0,,,,
42,4.200000e-11,0.000000e+00,42 Coin,0
404,5.320000e-04,1.065855e-03,404Coin,0
1337,3.141593e-01,2.957551e-02,EliteCoin,0
BTC,2.100000e-05,1.810842e-05,Bitcoin,1
ETH,0.000000e+00,1.087731e-04,Ethereum,1
...,...,...,...,...
ZEPH,2.000000e-03,2.020225e-03,ZEPHYR,0
GAP,2.500000e-04,1.508199e-05,Gapcoin,0
BDX,1.400223e-03,9.901351e-04,Beldex,1


In [370]:
# Create a hvplot.scatter plot using x="TotalCoinsMined" and y="TotalCoinSupply".
# YOUR CODE HERE
fig2 = X_supplied_mined_scaled_df.hvplot.scatter(x="TotalCoinsMined", y="TotalCoinSupply",by="class",hover_cols ="CoinName")
fig2


:NdOverlay   [class]
   :Scatter   [TotalCoinsMined]   (TotalCoinSupply,CoinName)

Running K-Means with `k=4`